### Initial setup

In [1]:
import cv2
import os
import json
from utils import *

images_dir = './samples'
input = 'input.json'
output = 'output.json'

### Lego detection

In [34]:
# ----------------------------------------------

with open(input, 'r') as json_file:
    data = json.load(json_file)

image_paths = data['image_files']
image_index = 40

image = cv2.imread(os.path.join(images_dir, image_paths[image_index]))
image = cv2.resize(image, (0, 0), fx = 0.15, fy = 0.15)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

complexity, _ = get_image_complexity(gray)

# ----------------------------------------------

hue_tolerance = 20
saturation_tolerance = 90
value_tolerance = 90
blur_kernel = (5, 5)
if complexity == 2:
    hue_tolerance = 20
    saturation_tolerance = 90
    value_tolerance = 120
    blur_kernel = (5, 5)
elif complexity == 3:
    hue_tolerance = 20
    saturation_tolerance = 110
    value_tolerance = 140
    blur_kernel = (11, 11)

bg_median_mask = background_median(hsv, hue_tolerance, saturation_tolerance, value_tolerance, blur_kernel)
bg_median = cv2.bitwise_and(image, image, mask=bg_median_mask)

blur_kernel = (7, 7)
low_threshold = 100
high_threshold = 200
if complexity == 2:
    blur_kernel = (5, 5)
    low_threshold = 70
    high_threshold = 150
elif complexity == 3:
    blur_kernel = (11, 11)
    low_threshold = 90
    high_threshold = 200

canny_mask = canny_edges(gray, blur_kernel, low_threshold, high_threshold)
canny = cv2.bitwise_and(image, image, mask=canny_mask)

combined_mask = cv2.bitwise_or(bg_median_mask, canny_mask)
final_mask = combined_mask
#if complexity == 'Complex':
#    blur_kernel = (3, 3)
#    bb_size = 100
#    iterations = 1
#    grabcut_mask = grab_cut(image, blur_kernel, bb_size, iterations)
#    grabcut = cv2.bitwise_and(image, image, mask=grabcut_mask)
#    final_mask = cv2.bitwise_and(combined_mask, grabcut_mask)
result = cv2.bitwise_and(image, image, mask=final_mask)

# ----------------------------------------------

contours, _ = cv2.findContours(final_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = filter_contours(contours)
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 255, 0), 2)

num_lego_pieces = len(contours)
print("Number of LEGO pieces detected:", num_lego_pieces)

# ----------------------------------------------

cv2.imshow('Image', image)
#if complexity == 'Complex':
#    cv2.imshow('GrabCut', grabcut)
cv2.imshow('Background Median', bg_median)
cv2.imshow('Canny', canny)
cv2.imshow('Final mask', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

# ----------------------------------------------

Number of LEGO pieces detected: 9
